In [1]:
import math
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class Implicit(nn.Module):
  def __init__(self, fun, dimension, radius=1):
    super(Implicit, self).__init__()

    d_in = dimension
    dims = [512, 512, 512, 512, 512, 512, 512, 512]
    beta = 100
    skip_in = [4]
    radius_init = radius

    dims = [d_in] + dims + [1]

    self.num_layers = len(dims)
    self.skip_in = skip_in

    for layer in range(0, self.num_layers - 1):

      if layer + 1 in skip_in:
        out_dim = dims[layer + 1] - d_in
      else:
        out_dim = dims[layer + 1]

      lin = nn.Linear(dims[layer], out_dim)

      if layer == self.num_layers - 2:
        torch.nn.init.normal_(lin.weight, mean=np.sqrt(np.pi) / np.sqrt(dims[layer]), std=0.00001)
        torch.nn.init.constant_(lin.bias, -radius_init)
      else:
        torch.nn.init.constant_(lin.bias, 0.0)
        torch.nn.init.normal_(lin.weight, 0.0, np.sqrt(2) / np.sqrt(out_dim))

      setattr(self, "lin" + str(layer), lin)

    self.activation = nn.Softplus(beta=beta)
    self.fun = fun


  def forward(self, input):
    x = input
    for layer in range(0, self.num_layers - 1):
      lin = getattr(self, "lin" + str(layer))
      if layer in self.skip_in:
        x = torch.cat([x, input], -1) / np.sqrt(2)
      x = lin(x)
      if layer < self.num_layers - 2:
        x = self.activation(x)
    fun_sign = torch.tanh(0.1*self.fun(input))
    fun_sign = fun_sign.reshape(x.shape)
    x = x * fun_sign
    return x

In [ ]:
def grad(y, x, grad_outputs=None):
  if grad_outputs is None:
    grad_outputs = torch.ones_like(y)
  grad = torch.autograd.grad(y, [x], grad_outputs=grad_outputs, create_graph=True)[0]
  return grad

def div(y, x):
  div = 0.
  for i in range(y.shape[-1]):
    div += torch.autograd.grad(y[..., i], x, torch.ones_like(y[..., i]), create_graph=True)[0][..., i:i+1]
  return div

def laplacian(y, x):
  grad = grad(y, x)
  return div(grad, x)

def pLaplacian(y, x, p=2):
  g = grad(y, x)
  g_n = torch.linalg.norm(g, 2, dim=1)
  g_n = g_n**(p-2)
  g_n = torch.reshape(g_n, (g.shape[0],1))
  g = g_n * g
  return div(g, x)

In [4]:
# Uniform distribution in 2d
def uniform_data(num_samples, x_bounds, y_bounds, device='cpu'):
  x_min, x_max = x_bounds
  y_min, y_max = y_bounds
  u_x = torch.FloatTensor(num_samples, 1).uniform_(x_min, x_max)
  u_y = torch.FloatTensor(num_samples, 1).uniform_(y_min, y_max)
  u = torch.cat((u_x, u_y), dim=-1)  
  u = u.to(device)
  return u

In [ ]:
def trainPPoisson(num_iters, fun, xbounds, ybounds, p=2, device='cpu'):
  model = Implicit(fun=fun, dimension=2).to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

  n_samples = 256
  loss = 0

  # Train network
  for i in range(0, num_iters):
    # Uniform samples
    u = uniform_data(n_samples, xbounds, ybounds, device)
    u.requires_grad = True
    
    model.train()
    optimizer.zero_grad()
    
    f_d = model(u)
    
    # p-Laplacian
    lap = pLaplacian(f_d, u, p)
    lap_loss = torch.mean((lap+1)**2)
    loss = lap_loss

    loss.backward()

    optimizer.step()
  
  return model

In [ ]:
def trainEikonal(num_iters, fun, xbounds, ybounds, device='cpu'):
  model = Implicit(fun=fun, dimension=2).to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

  n_samples = 1024
  loss = 0

  # Train network
  for i in range(0, num_iters):
    # Uniform samples
    u = uniform_data(n_samples, xbounds, ybounds, device)
    u.requires_grad = True
    
    model.train()
    optimizer.zero_grad()

    f_d = model(u)
    
    g_d = grad(f_d, u)
    g_norm = (g_d.norm(2, dim=1) - 1)**2
    eik_loss = torch.mean(g_norm)
    loss = eik_loss

    loss.backward()

    optimizer.step()
  
  return model

In [7]:
# The implicit surface (a unit disk)
def f(p):
  return 1.0 - p[:,0]**2 - p[:,1]**2

In [8]:
# For visualization
def showZeroLevel(x, y, f):
  plt.figure(figsize=(8,4))
  h = plt.contour(x, y, f, levels=[0.0], colors='b')
  h.ax.axis('equal')
  plt.title('Zero level-set')
  plt.show()

def showContourPlot(x, y, f): 
  plt.figure(figsize=(8,4))
  h = plt.contourf(x, y, f)
  h.ax.axis('equal')
  plt.title('Filled Contour Plot')
  plt.show()

In [ ]:
# Create a grid from torch tensors
def torchLinearGrid(xbounds, ybounds, grid_res, device='cpu'):
  xmin = xbounds[0]
  xmax = xbounds[1]
  ymin = ybounds[0]
  ymax = ybounds[1]

  dx = xmax - xmin
  dy = ymax - ymin

  resx = grid_res[0]
  resy = grid_res[1]

  x = torch.arange(xmin, xmax, step=dx/float(resx))
  y = torch.arange(ymin, ymax, step=dy/float(resy))

  xx, yy = torch.meshgrid(x, y, indexing='ij')

  xx = xx.to(device)
  yy = yy.to(device)

  dimg = resx * resy
  xy = torch.stack((xx, yy), dim=-1).reshape(dimg,2)

  return xy

In [10]:
def torchLinearSampling(model, xy):
  d = model(xy)
  return d

In [11]:
def saveVTK(filename, xy, res, field):
  resx, resy = res
  resz = 1
  
  # set the z coord to 0 
  xyz = torch.zeros((xy.shape[0], 3))
  xyz[:,0:2] = xy

  field_title = 'VALUE'

  with open(filename, 'w') as f:
    f.write('# vtk DataFile Version 3.0\n')
    f.write('vtk output\n')
    f.write('ASCII\n')
    f.write('DATASET STRUCTURED_GRID\n')
    f.write('DIMENSIONS ' + str(resx) + ' ' + str(resy) + ' ' + str(resz) +'\n')
    f.write('POINTS ' + str(resx*resy*resz) + ' double\n')

    np.savetxt(f, xyz.detach().cpu().numpy())
    
    f.write('\n\n')

    f.write('POINT_DATA ' + str(resx*resy*resz) + '\n')
    f.write('SCALARS ' + field_title + ' double' + '\n')
    f.write('LOOKUP_TABLE default\n')
        
    np.savetxt(f, field.detach().cpu().numpy())
    f.write('\n')

In [12]:
# Domain boundary
xbounds = (-2.0, 2.0)
ybounds = (-2.0, 2.0)

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [14]:
xy = torchLinearGrid(xbounds, ybounds, (64, 64), device=device)

In [15]:
yt = f(xy)

In [16]:
saveVTK('2d_disk_rfunction.vtk', xy, (64,64), yt)

In [17]:
# p-Poisson problem, p = 8
max_iteration = 15000
p = 8
model_p8 = trainPPoisson(max_iteration, fun=f, xbounds=xbounds, ybounds=ybounds, p=p, device=device)

In [18]:
yt_pLap_p8 = model_p8(xy)

In [19]:
saveVTK('2d_disk_pLap_p8.vtk', xy, (64,64), yt_pLap_p8)

In [20]:
# Eikonal problem
max_iteration = 15000
model_eik = trainEikonal(max_iteration, fun=f, xbounds=xbounds, ybounds=ybounds, device=device)

In [21]:
yt_eik = model_eik(xy)

In [22]:
saveVTK('2d_disk_eikonal.vtk', xy, (64,64), yt_eik)